Adapted from https://towardsdatascience.com/pydantic-688e897cfd3a

In [1]:
import datetime
from decimal import Decimal
from typing import List, NewType, Optional

# pip install pydantic
from pydantic import BaseModel, Field, parse_file_as, validator

In [2]:
# We create a new type for the ID of a person, because 
# PersonId is so much more meaningful than just int
PersonId = NewType("PersonId", int)

class Person(BaseModel):
    id: PersonId
    name: str
    bank_account: Decimal
    birthdate: datetime.date
    friends: Optional[List[PersonId]] # this attributed can be omitted
    hobbies: List[str] = Field(default_factory=list) # default value -> empty list
        
    @validator('bank_account')
    def validate_bank_account(cls, value):
        if value < 0:
            raise ValueError("Negative bank_account")
        else:
            return value

In [3]:
people = [
{"id": 0, "name": "Anna", "friends": None, "birthdate": "1992-01-15", "bank_account": 12.3},
{"id": 1, "friends": [3], "birthdate": "1962-12-31", "bank_account": 0.1},
{"id": 2, "name": "Charlie", "friends" : [0, 2], "birthdate": "1992-02-999", "bank_account": 9007199254740993.0},
{"id": 3, "name": "Martin", "friends": [1, 2, 3], "birthdate": "1990-04-28", "bank_account": -5}
]

In [4]:
people[0]

{'id': 0,
 'name': 'Anna',
 'friends': None,
 'birthdate': '1992-01-15',
 'bank_account': 12.3}

In [5]:
person_0 = Person(**people[0])
person_0

Person(id=0, name='Anna', bank_account=Decimal('12.3'), birthdate=datetime.date(1992, 1, 15), friends=None, hobbies=[])

In [6]:
person_0.name

'Anna'

In [7]:
# Error, because people[1] is missing the "name" key, 
# and it is not Optional (like "friends")
person_1 = Person(**people[1])
person_1

ValidationError: 1 validation error for Person
name
  field required (type=value_error.missing)

In [8]:
# Error, because birthdate doesn't have a valid format
person_2 = Person(**people[2])
person_2

ValidationError: 1 validation error for Person
birthdate
  invalid date format (type=value_error.date)

In [9]:
# Error, because bank_account has a negative value (didn't pass the validator)
person_3 = Person(**people[3])
person_3

ValidationError: 1 validation error for Person
bank_account
  Negative bank_account (type=value_error)

# Reading JSON config

In [10]:
from pathlib import Path

### Valid config

In [11]:
print(Path('resources/people_valid.json').read_text())

[
{"id": 0, "name": "Anna", "friends": [1, 2, 3], "birthdate": "1992-01-15", "bank_account": 12.3},
{"id": 1, "name": "Nicole", "friends": [3], "birthdate": "1962-12-31", "bank_account": 0.1},
{"id": 2, "name": "Charlie", "friends" : [0, 2], "birthdate": "1992-02-28", "bank_account": 9007199254740993.0},
{"id": 3, "name": "Martin", "friends": null, "birthdate": "1990-04-28", "bank_account": 9007199254740993}
]


In [12]:
valid_config = parse_file_as(List[Person], 'resources/people_valid.json')
valid_config

[Person(id=0, name='Anna', bank_account=Decimal('12.3'), birthdate=datetime.date(1992, 1, 15), friends=[1, 2, 3], hobbies=[]),
 Person(id=1, name='Nicole', bank_account=Decimal('0.1'), birthdate=datetime.date(1962, 12, 31), friends=[3], hobbies=[]),
 Person(id=2, name='Charlie', bank_account=Decimal('9007199254740992.0'), birthdate=datetime.date(1992, 2, 28), friends=[0, 2], hobbies=[]),
 Person(id=3, name='Martin', bank_account=Decimal('9007199254740993'), birthdate=datetime.date(1990, 4, 28), friends=None, hobbies=[])]

### Invalid config

In [13]:
print(Path('resources/people_invalid.json').read_text())

[
{"id": 0, "name": "Anna", "friends": [1, 2, 3], "birthdate": "1992-01-15", "bank_account": 12.3},
{"id": 1, "friends": [3], "birthdate": "1962-12-31", "bank_account": 0.1},
{"id": 2, "name": "Charlie", "friends" : [0, 2], "birthdate": "1992-02-999", "bank_account": 9007199254740993.0},
{"id": 3, "name": "Martin", "friends": null, "birthdate": "1990-04-28", "bank_account": 9007199254740993}
]


In [14]:
invalid_config = parse_file_as(List[Person], 'resources/people_invalid.json')
invalid_config

ValidationError: 2 validation errors for ParsingModel[List[__main__.Person]]
__root__ -> 1 -> name
  field required (type=value_error.missing)
__root__ -> 2 -> birthdate
  invalid date format (type=value_error.date)